In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Accounts')

+------------+--------+
| account_id | income |
+------------+--------+
|     3      | 108939 |
|     2      | 12747  |
|     8      | 87709  |
|     6      | 91796  |
+------------+--------+


### Calculate the number of bank accounts of each salary category. The salary categories are:

- "Low Salary": All the salaries strictly less than $20000.
- "Average Salary": All the salaries in the inclusive range [$20000, $50000].
- "High Salary": All the salaries strictly greater than $50000.
#### *The result table must contain all three categories. If there are no accounts in a category, then report 0.*

```
+----------------+----------------+
| category       | accounts_count |
+----------------+----------------+
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |
+----------------+----------------+
Explanation: 
Low Salary: Account 2.
Average Salary: No accounts.
High Salary: Accounts 3, 6, and 8.
```

In [11]:
%%sql 

SELECT 
    (CASE WHEN income < 20000 THEN 'Low Salary'
    WHEN income >= 20000 AND INCOME < 50000 THEN 'Average Salary'
    ELSE 'High Salary' END) AS category
FROM Accounts

category
High Salary
Low Salary
High Salary
High Salary


In [53]:
%%sql 

SELECT
    category,
    COUNT(category) AS accounts_count
FROM (
    SELECT 
        (CASE WHEN income < 20000 THEN 'Low Salary'
        WHEN income >= 20000 AND income < 50000 THEN 'Average Salary'
        ELSE 'High Salary' END) AS category
    FROM Accounts
) AS count_table
GROUP BY category;


category,accounts_count
High Salary,3
Low Salary,1


In [30]:
%%sql 

SELECT 'Low Salary' AS category
UNION 
SELECT 'Average Salary' AS category
UNION
SELECT 'High Salary' AS category

category
Low Salary
Average Salary
High Salary


In [55]:
%%sql 

SELECT 
    category 
FROM (
    SELECT 'Low Salary' AS category
    UNION 
    SELECT 'Average Salary' AS category
    UNION
    SELECT 'High Salary' AS category
) AS definitions_table;

category
Low Salary
Average Salary
High Salary


In [65]:
%%sql

SELECT
    category_table.category,
    count_table.accounts_count
FROM (
    SELECT 
        category 
    FROM (
        SELECT 'Low Salary' AS category
        UNION 
        SELECT 'Average Salary' AS category
        UNION
        SELECT 'High Salary' AS category
    ) AS t1
) AS category_table
LEFT JOIN (
    SELECT
    category,
    COUNT(category) AS accounts_count
    FROM (
        SELECT 
            (CASE WHEN income < 20000 THEN 'Low Salary'
            WHEN income >= 20000 AND income < 50000 THEN 'Average Salary'
            ELSE 'High Salary' END) AS category
        FROM Accounts
    ) AS t2
    GROUP BY category
) AS count_table
ON category_table.category = count_table.category

category,accounts_count
Low Salary,1
Average Salary,None
High Salary,3


In [64]:
%%sql

SELECT
    category_table.category,
    IFNULL(count_table.accounts_count, 0) AS accounts_count
FROM (
    SELECT 
        category 
    FROM (
        SELECT 'Low Salary' AS category
        UNION 
        SELECT 'Average Salary' AS category
        UNION
        SELECT 'High Salary' AS category
    ) AS t1
) AS category_table
LEFT JOIN (
    SELECT
    category,
    COUNT(category) AS accounts_count
    FROM (
        SELECT 
            (CASE WHEN income < 20000 THEN 'Low Salary'
            WHEN income >= 20000 AND income < 50000 THEN 'Average Salary'
            ELSE 'High Salary' END) AS category
        FROM Accounts
    ) AS t2
    GROUP BY category
) AS count_table
ON category_table.category = count_table.category

category,accounts_count
Low Salary,1
Average Salary,0
High Salary,3


# Using CTE

In [68]:
%%sql 

WITH category_cte AS(
    SELECT 
        category
    FROM (
        SELECT 'Low Salary' AS category
        UNION 
        SELECT 'Average Salary' AS category 
        UNION 
        SELECT 'High Salary' AS category 
    ) t1
), 
count_cte AS(
    SELECT 
        category,
        COUNT(category) AS accounts_count
    FROM (
        SELECT 
            (CASE WHEN income < 20000 THEN 'Low Salary'
            WHEN income >= 20000 AND income < 50000 THEN 'Average Salary'
            ELSE 'High Salary' END) AS category
        FROM Accounts
    ) t2 
    GROUP BY t2.category
)
SELECT 
    category_cte.category,
    count_cte.accounts_count
FROM category_cte
LEFT JOIN count_cte ON category_cte.category = count_cte.category;

category,accounts_count
Low Salary,1
Average Salary,None
High Salary,3


In [69]:
%%sql 

WITH category_cte AS(
    SELECT 
        category
    FROM (
        SELECT 'Low Salary' AS category
        UNION 
        SELECT 'Average Salary' AS category 
        UNION 
        SELECT 'High Salary' AS category 
    ) t1
), 
count_cte AS(
    SELECT 
        category,
        COUNT(category) AS accounts_count
    FROM (
        SELECT 
            (CASE WHEN income < 20000 THEN 'Low Salary'
            WHEN income >= 20000 AND income < 50000 THEN 'Average Salary'
            ELSE 'High Salary' END) AS category
        FROM Accounts
    ) t2 
    GROUP BY t2.category
)
SELECT 
    category_cte.category,
    IFNULL(count_cte.accounts_count, 0) AS accounts_count
FROM category_cte
LEFT JOIN count_cte ON category_cte.category = count_cte.category;

category,accounts_count
Low Salary,1
Average Salary,0
High Salary,3


# Using Pandas

In [70]:
accounts_query = %sql SELECT * FROM Accounts # type: ignore
accounts_df = accounts_query.DataFrame()

display(accounts_df)

,account_id,income
0,3,108939
1,2,12747
2,8,87709
3,6,91796
